# Forward kinematics of 2dof planar robots

## Case 1) Two revolute joints

<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/2d-2dof-revolute.png?raw=true" width=400 />


## Case 2) Revolute joint followed by prismatic joint

<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/2d-2dof-revolute-prismatic.png?raw=true" width=400 />



### Preliminaries - Installing KDL
API documentation for KDL:
- http://docs.ros.org/en/diamondback/api/kdl/html/python/
- http://docs.ros.org/en/indigo/api/orocos_kdl/html/namespaceKDL.html

In [ ]:
!git clone https://github.com/orocos/orocos_kinematics_dynamics.git
!apt install libeigen3-dev libcppunit-dev # Without sudo on colab
!mkdir orocos_kinematics_dynamics/orocos_kdl/build
%cd orocos_kinematics_dynamics/orocos_kdl/build
!cmake ..
!make
!make install
%cd ../../python_orocos_kdl
!git submodule update --init
!mkdir build
%cd build
!cmake ..
!make
!make install
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib
!ldconfig
!python3 ../tests/PyKDLtest.py

### Case 1

In [ ]:
import numpy as np
import doctest
import PyKDL as kdl

In [ ]:
class TwoR:
    ''' Represents a planar robotic arm RR
    '''

    def __init__(self, l1=2, l2=1):
        '''
        Arguments
        ---------
        l1, l2 : float
           Length of the two links, respectively.
        '''
        
        # Define a kinematic chain, by connecting segments
        arm = kdl.Chain()
        
        # The first segment is rotating about the z-axis, and its tip
        # is located at a distance of l1 in the x-direction.
        segment1 = kdl.Segment(joint=kdl.Joint(kdl.Joint.RotZ), 
                              f_tip = kdl.Frame(kdl.Rotation.Identity(), 
                                               kdl.Vector(l1, 0, 0)))
        arm.addSegment(segment1)
        
        # The second segment when added to the chain, will have its joint located
        # at the tip of the previous segment.
         
        ########################################################################
        # Your code here
        # Define the second segment and add it to the chain
        ########################################################################
        
        self.arm = arm
        
        # The forward kinematics solver
        self.fksolver = kdl.ChainFkSolverPos_recursive(self.arm)
        
        
    def fk(self, th1, th2):
        '''
        Implements the forward kinematics.
        
        Arguments
        ---------
        th1, th2 : float
           Angle in radians of the two degree of freedoms, respectively.
    
        Returns
        -------
        x : float
           The position in the global x-direction of the end-effector (tool point)
        y : float
           The position in the global y-direction of the end-effector (tool point)
        theta : float
           The orientation of the end-effector with respect to the positive global x-axis.
           The angle returned is in the range [-np.pi, np.pi]
       j : tuple with 2 elements
           The position of the joint between the two links
    
        Tests
        ------
        >>> l1, l2 = 2, 1
        >>> arm = TwoR(l1, l2)
        >>> # 1) End-effector pose at default position
        >>> x, y, th, j = arm.fk(0, 0)
        >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, th)
        '(3.00, 0.00, 0.00)'
        >>> # 2) End-effector pose at 90 degrees in both joints
        >>> x, y, th, j = arm.fk(np.pi/2, np.pi/2)
        >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, th)
        '(-1.00, 2.00, 3.14)'
        >>> # 3) End-effector pose at 0 degress in first joint and 90 degress in second
        >>> x, y, th, j = arm.fk(0, np.pi/2)
        >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, th)
        '(2.00, 1.00, 1.57)'
        >>> # 4) End-effector position is always inside a circle of radius l1+l2
        >>> poses = [arm.fk(th1_, th2_)
        ... for th1_ in np.arange(0, 2*np.pi, 0.2) 
        ... for th2_ in np.arange(0, 2*np.pi, 0.2)]
        >>> distances = np.array([np.sqrt(x_**2 + y_**2) for x_, y_, th_, j_ in poses])
        >>> max_radius = l1+l2 + 1e-12 # Add a small tolerance
        >>> np.any(distances > max_radius)
        False
        >>> # 5) Joint is always at constant distance from the origin
        >>> poses = [arm.fk(th1_, 0)
        ... for th1_ in np.arange(0, 2*np.pi, 0.2) ] 
        >>> distances = np.array([np.sqrt(j_[0]**2 + j_[1]**2) for x_, y_, th_, j_ in poses])
        >>> np.any(np.abs(distances - l1) > 1e-12)
        False
        
        '''

        thetas = kdl.JntArray(2)
        thetas[0] = th1
        thetas[1] = th2
        
        ee_frame = kdl.Frame()
        res = self.fksolver.JntToCart(thetas, ee_frame)
        if res < 0:
            raise RuntimeError("Forward kinematics solver failed")
        # The center of the end effector frame
        tcp = ee_frame.p
        
        # The position of the joint
        j_frame = kdl.Frame()
        res = self.fksolver.JntToCart(thetas, j_frame, 1)
        if res < 0:
            raise RuntimeError("Forward kinematics solver failed")
        jc = j_frame.p
        
        # Get rotation of end effector
        x_e = ee_frame.M * kdl.Vector(1, 0, 0)
        theta = np.arctan2(x_e.y(), x_e.x())
        
        return tcp.x(), tcp.y(), theta, (jc.x(), jc.y())
        

## Run doctests
If tests pass, no output is generated.

In [ ]:
# Case 1)
doctest.run_docstring_examples(TwoR.fk, globals())

## Case 2)

In [ ]:
class OneROneP:
    ''' Represents a manipulator with one revolute joint and one prismatic joint. 
    '''
    
    def __init__(self, l1=2):
        '''
        Arguments
        ---------
        l1 : float
           Length of the first link.
         '''
        # Define a kinematic chain, by connecting segments
        arm = kdl.Chain()
        
        #####################################################################
        # Your code here
        # Define the kinematic chain
        #####################################################################
        
        self.arm = arm
        
        # The forward kinematics solver
        self.fksolver = kdl.ChainFkSolverPos_recursive(self.arm)
        
        
        
    def fk(self, th1, th2):
        '''
        Implements the forward kinematics of a robot with one revolute joint and one prismatic. 
    
        Arguments
        ---------
        th1 : float
           Angle in radians of the first degree of freedom.
        th2 : float
           Displacement in meter of the second degree of freedom.

        Returns
        -------
        x : float
           The position in the global x-direction of the end-effector (tool point)
        y : float
           The position in the global y-direction of the end-effector (tool point)
        theta : float
           The orientation of the end-effector with respect to the positive global x-axis

        Tests
        ------
        >>> l1 = 2
        >>> arm = OneROneP(l1=l1)
        >>> # 1) End-effector pose at default position
        >>> "(%0.2f, %0.2f, %0.2f)" %arm.fk(0, 0)
        '(2.00, 0.00, 0.00)'
        >>> # 2) End-effector pose at 90 degrees in first joint and 0.6m in second
        >>> "(%0.2f, %0.2f, %0.2f)" %arm.fk(np.pi/2, 0.6)
        '(0.00, 2.60, 1.57)'
        >>> # 3) End-effector orientation is always the same as the angle of the first dof
        >>> angles = np.array( [th1_ for th1_ in np.arange(-np.pi+0.1, np.pi, 0.2)
        ... for th2_ in np.arange(-1, 1, 0.2)])
        >>> poses = [arm.fk(th1_, th2_)
        ... for th1_ in np.arange(-np.pi+0.1, np.pi, 0.2)
        ... for th2_ in np.arange(-1, 1, 0.2)]
        >>> orientations = np.array([th_ for x_, y_, th_ in poses])
        >>> np.any(np.abs(angles-orientations) > 1e-12)
        False
        '''
        
        thetas = kdl.JntArray(2)
        thetas[0] = th1
        thetas[1] = th2
        
        ee_frame = kdl.Frame()
        res = self.fksolver.JntToCart(thetas, ee_frame)
        if res < 0:
            raise RuntimeError("Forward kinematics solver failed")
    
        # The center of the end effector frame
        tcp = ee_frame.p
        
        #####################################################################
        # Your code here
        # Find the orientation of the end effector
        # Return the correct values
        #####################################################################
        

## Run doctests
If tests pass, no output is generated.

In [ ]:
# Case 2)
doctest.run_docstring_examples(OneROneP.fk, globals())

## Visualize the work space of the 2R robot

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
arm = TwoR()
th1 = np.arange(0, 2*np.pi, 0.1)
th2 = np.arange(-np.pi, np.pi, 0.1)
xythetaj =[ arm.fk(th1_, th2_) for th1_ in th1 for th2_ in th2] 
xytheta = np.array([ (x_, y_, th_) for x_, y_, th_, j_ in xythetaj])
df = pd.DataFrame(data=np.reshape(xytheta, (-1,3)), columns=['x', 'y', 'theta'])

In [ ]:
fig = px.scatter_3d(df, x='x', y='y', z='theta')
camera = dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=4)
)
fig.update_scenes(camera_projection_type="orthographic")
fig.update_layout(scene_camera=camera)
fig.show()



## Visualize movement of the manipulator

In [ ]:
poses = [ arm.fk(th1_, th2_) for th1_, th2_ in zip(th1, th2)]
endeff_trajectory = np.array([ [x_, y_]  for x_, y_, th_, j_ in poses])
joint_trajectory = np.array([ j_  for x_, y_, th_, j_ in poses])

In [ ]:
fig = go.Figure(
    data=[go.Scatter(x=[0, joint_trajectory[0,0]], y=[0, joint_trajectory[0,1]], 
          name="First link", mode="lines",
          line=dict(width=6, color="blue")),
          go.Scatter(x=[joint_trajectory[0,0], endeff_trajectory[0,0]], 
                     y=[joint_trajectory[0,1], endeff_trajectory[0,1]], 
          name="Second link", mode="lines",
          line=dict(width=5, color="red")),
         go.Scatter(x=joint_trajectory[:,0], y=joint_trajectory[:,1], 
          name="Joint trajectory", mode="lines",
          line=dict(width=1, color="lightblue")),
          go.Scatter(x=endeff_trajectory[:,0], y=endeff_trajectory[:,1], 
          name="End-point trajectory", mode="lines",
          line=dict(width=1, color="red"))],
    layout=go.Layout( width=700, height=600,
        xaxis=dict(range=[-4, 4], autorange=False),
        yaxis=dict(range=[-4, 4], autorange=False),
        title="End-effector trajectory",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=[0, xj_], y=[0, yj_]),
                          go.Scatter(x=[xj_, xe_], y=[yj_, ye_])]) 
            for xj_, yj_, xe_, ye_ in np.hstack((joint_trajectory, endeff_trajectory))]
)

fig.show()